In [241]:
pip install imblearn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
df = pd.read_csv('/content/file3.csv', low_memory = False)

In [5]:
df.head()

,Unnamed: 0,extended,crit3,suicide,property,targtype1,attacktype1,gname,weaptype1,country_txt,success
0,39,0,0,0,0,4,9,New People's Army (NPA),13,Philippines,0
1,169,0,0,0,0,4,9,New People's Army (NPA),13,Philippines,1
2,226,0,1,0,1,6,3,Palestinians,6,Turkey,1
3,364,0,1,0,1,17,2,Irish Republican Army (IRA),8,United Kingdom,0
4,365,0,1,0,0,14,2,Irish Republican Army (IRA),5,United Kingdom,1


In [0]:
#embedding the gname 
le_gname = preprocessing.LabelEncoder()
le_country = preprocessing.LabelEncoder()
le_gname.fit(df['gname'])
le_country.fit(df['country_txt'])

array_g = np.array(le_gname.transform(df['gname']))
array_g = pd.DataFrame({'gname_embedded': array_g[:]})

array_c= np.array(le_country.transform(df['country_txt']))
array_c = pd.DataFrame({'country_embedded': array_c[:]})

In [0]:
df_embedded = df.assign(gname = array_g['gname_embedded'])
df_embedded = df_embedded.assign(country_txt = array_c['country_embedded'])

In [8]:
df_embedded = df_embedded.drop('Unnamed: 0', axis = 1)
df_embedded

,extended,crit3,suicide,property,targtype1,attacktype1,gname,weaptype1,country_txt,success
0,0,0,0,0,4,9,22,13,30,0
1,0,0,0,0,4,9,22,13,30,1
2,0,1,0,1,6,3,24,6,39,1
3,0,1,0,1,17,2,13,8,41,0
4,0,1,0,0,14,2,13,5,41,1
...,...,...,...,...,...,...,...,...,...,...
57673,0,0,0,0,4,2,14,5,16,0
57674,0,0,0,-9,4,3,4,6,32,1
57675,0,1,0,0,17,2,29,5,0,1
57676,0,0,0,-9,4,2,4,5,32,1


In [0]:
labels = np.array(df_embedded['success'])
df_embedded = df_embedded.drop('success', axis = 1)
feature_list = list(df_embedded.columns)

In [10]:
#feature_list.remove('Unnamed: 0')
print(labels, len(feature_list))

[0 1 1 ... 1 1 1] 9


In [11]:
df_embedded

,extended,crit3,suicide,property,targtype1,attacktype1,gname,weaptype1,country_txt
0,0,0,0,0,4,9,22,13,30
1,0,0,0,0,4,9,22,13,30
2,0,1,0,1,6,3,24,6,39
3,0,1,0,1,17,2,13,8,41
4,0,1,0,0,14,2,13,5,41
...,...,...,...,...,...,...,...,...,...
57673,0,0,0,0,4,2,14,5,16
57674,0,0,0,-9,4,3,4,6,32
57675,0,1,0,0,17,2,29,5,0
57676,0,0,0,-9,4,2,4,5,32


In [0]:
df_embedded = np.array(df_embedded)

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(df_embedded, labels, test_size = 0.25, random_state = 42)

In [22]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (43258, 9)
Training Labels Shape: (43258,)
Testing Features Shape: (14420, 9)
Testing Labels Shape: (14420,)


In [0]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 42)

In [0]:
smt = SMOTE()
train_features, train_labels = smt.fit_sample(train_features, train_labels)

In [25]:
rf.fit(train_features, train_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
smt_test = SMOTE()
test_features, test_labels = smt_test.fit_sample(test_features, test_labels)

In [27]:
y_pred=rf.predict(test_features)
y_pred

array([0, 0, 0, ..., 0, 1, 0])

In [28]:
print("Accuracy:",metrics.accuracy_score(test_labels, y_pred))

Accuracy: 0.8312761904761905
